# Infer Neuro Behavioral Ontology terms from text

TODO: Describe the following as diagrams.

Two big steps:
* Use zero-shot learning as an NER to extract behavioral terms from specific sections of research papers.
* Use vector databases to ground the terms to specific concepts in the [Neuro Behavioral Ontology](https://www.ebi.ac.uk/ols4/ontologies/nbo?tab=classes).


# Pipeline

In [1]:
import pprint
from pathlib import Path
import os 


from repo_secrets import OPENAI_API_KEY
from repo_secrets import QDRANT_API_KEY

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["QDRANT_API_KEY"] = QDRANT_API_KEY

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Methods text from the paper

In [85]:
co_housing_text = (
"""Co-housing
Pup-naive C57Bl/6 virgin female mice were bred and raised at NYU School of Medicine and kept isolated from dams and pups until used for these studies when approximately eight weeks old. For experiments where viral injections were performed, we first allowed two weeks for viral expression before animals were used in experiments. Dams were initially pre-screened to ensure they behaved maternally, meaning that they retrieved pups and built nests; about 1% of dams did not retrieve pups and these animals were not used for co-housing. Naive virgins were initially pre-screened for retrieval or pup mauling before co-housing; around 5% of the naive virgins retrieved at least one pup or mauled pups during pre-screening and these mice were excluded from subsequent behavioural studies.

Co-housing of a virgin female with a mother and litter was conducted for 4–6 consecutive days in 80 × 40 × 50 cm plastic home cages. The floor was covered with abundant bedding material, food pellets and a pack of hydrogel for hydration placed in a corner of the bin and refreshed daily. Nesting material was also placed in the cage. We first placed the dam and her postnatal day 1 (P1) litter in the cage. After the dam was acclimatized for ~30 min, we introduced the virgin female with a tail mark for identification. Well-being of the adult mice and pups was monitored at least twice a day. A surveillance infrared camera system (Blackrock Microsystems) was positioned ~100 cm above the home cage to capture the entire surface. An ultrasonic microphone (Avisoft) was placed in the corner of the cage, ~10 cm above the nest. Two initial cages had a second camera placed on the side but these videos were not analysed for these experiments. For studies of spontaneous pup retrieval by dams and the influence of co-housing, some dams were singly housed with their litter but not with other adults.

In cases where co-housing was done only between a virgin female and pups (Fig. 1c), the pups were returned to the donor mother every 12 h (for at least 48 h) and immediately replaced with new pups. This was done to ensure that they stay alive and healthy despite not being fed during co-housing with the virgin. The procedure was repeated throughout the duration of the co-housing27.
"""
)

pup_retrieval_text = (
"""
Pup retrieval testing
This test was used for the initial screening of dams and virgin female mice. In addition, outside of the spontaneous home cage behaviours, we specifically monitored pup retrieval every 24 h by the virgin females. We placed the female mouse to be tested in a behavioural arena (38 × 30 × 15 cm) containing bedding and nesting material; the female was alone, without contact with other animals. Each animal was given 20 min to acclimatize before each testing session began. The entire litter (ranging from 3 to 7 P1–4 pups) were grouped in a corner of the arena and covered with nesting material, and the adult female given an additional 2 min of acclimatization (pup group size did not affect retrieval behaviour; Extended Data Fig. 2c). One pup was removed from the nest and placed in an opposite corner of the arena. The experimental female was given 2 min per trial to retrieve the displaced pup and return it back to the nest; if the displaced pup was not retrieved within 2 min, the pup was returned to the nest and the trial was scored as a failure. If the pup was successfully retrieved, the time to retrieval was recorded and the trial was scored as a success. Another pup was then taken out of the nest, placed away from the nest (varying the position of the isolated pup relative to the nest from trial to trial), and the next trial was begun. After ten trials, pups were placed back into their home cage with their dam. We used an ultrasonic microphone (Avisoft) to verify that isolated pups vocalized during testing.

We reported probability of retrieving out of ten trials. Reliable retrieval was defined as having at least two out of ten successful trials. We used two-way ANOVA and Sidak’s multiple-comparison test corrections to compare probability of retrieving in each group over days, and Student’s t-test to compare the day of retrieval onset for each group.
"""
)

video_and_audio_analysis = (
"""
Video and audio analysis
Video and audio recordings were synchronized with the neuronal recordings, and then analysed with Adobe Audition and Avisoft. For video recordings we used the BORIS suite for scoring of behavioural observations. Three separate teams of independent scorers (two scorers from the Sullivan laboratory, three scorers from the Carcea laboratory and four scorers from the Froemke laboratory) were trained in a similar way on how to identify relevant individual and social behaviours during co-housing, and then scored the videos blind to the conditions. The results from each raster were compared and compiled, and results from each lab were cross-validated. Nest entry was considered the moment when the head of the animal entered the nest. Nest exit was considered the time when the rear of the animal left the nest. We used two-way ANOVA and Sidak’s multiple-comparison test to compare pup retrieval rates and time in nest across days for each group.

Any event in which the dam chased the virgin towards the nest was identified as a shepherding event (that is, where distance from start to nest was greater than distance from end to nest). To determine the distance from nest during shepherding, we measured the distance from the bottom left corner of the cage to the position of the snout of the mouse, and to the position of the nest center. We then calculated distance from the virgin to nest. In cases of physical contact, start of shepherding was considered to be the moment when the dam made contact with the virgin, and the end of shepherding was the moment when the virgin stopped running. In some cases (especially later into co-housing), we noticed that virgins started running as soon as they noticed the dam approaching; in those cases, the start of shepherding was considered to be the moment when the virgins started running after the dam’s approach. For Fig. 1i, we used paired t-tests to compare distance from start of shepherding to nest with the distance from end of shepherding to nest. For Fig. 1j, we used one-sample Student’s t-tests to determine if the daily frequency of shepherding was higher than 0.2 events per h (which was the average rate of dam–virgin chases in absence of pups). Audio recordings were processed in Adobe Audition, and isolation or distress calls were distinguished from adult calls and wriggling calls on the basis of the characteristic statistics (bout rate of 4–8 Hz and frequencies of 40–90 kHz).
"""
)


observation_of_experienced_retrievers = (
"""
Observation of experienced retrievers
We first confirmed that virgins did not retrieve and dams retrieved at 100% at baseline. The exposures were done in standard behavioural arena (38 × 30 × 15 cm). The virgin and dam were acclimatized for 20 min, then the nest with pups was transferred to this arena. After another 5–10 min, we manually isolated one pup at a time so that the dam would retrieve the pup back into the nest. We repeated this for ten times per session. In the experiments where either a transparent or an opaque divided the cage, the two adult animals were acclimatized on opposite sides of the barriers. After exposure, the adult animals were separated and the virgins were tested for pup retrieval 30 min later, as described above. As the preparation for testing and the acclimatization to the testing cage also took 30 min, this amounted to a total 60-min interval between virgin observation and testing of responses to isolated pups. The exposure was repeated for four sessions (one per day). A virgin that retrieved at least once during the four days of observation was considered as having acquired pup retrieval behaviour. We used chi-square exact tests to compare retrieval between conditions: wild-type mice with no barrier, wild-type mice with transparent barrier, wild-type mice with opaque barrier, and OXTR-KO virgins with transparent barrier.  
"""
)

methods_text = (
    f"{co_housing_text} {pup_retrieval_text} {video_and_audio_analysis} {observation_of_experienced_retrievers}" 
)

pprint.pprint(methods_text)

('Co-housing\n'
 'Pup-naive C57Bl/6 virgin female mice were bred and raised at NYU School of '
 'Medicine and kept isolated from dams and pups until used for these studies '
 'when approximately eight weeks old. For experiments where viral injections '
 'were performed, we first allowed two weeks for viral expression before '
 'animals were used in experiments. Dams were initially pre-screened to ensure '
 'they behaved maternally, meaning that they retrieved pups and built nests; '
 'about 1% of dams did not retrieve pups and these animals were not used for '
 'co-housing. Naive virgins were initially pre-screened for retrieval or pup '
 'mauling before co-housing; around 5% of the naive virgins retrieved at least '
 'one pup or mauled pups during pre-screening and these mice were excluded '
 'from subsequent behavioural studies.\n'
 '\n'
 'Co-housing of a virgin female with a mother and litter was conducted for 4–6 '
 'consecutive days in 80 × 40 × 50 cm plastic home cages. The floor

## Zero-short Named Entity Recognition with LLMs

In [100]:
from utils.beahvior_meatdata_extraction import extract_behavior_descriptions_from_prompt

instructions_prompt = (
    "You are a neuroscience researcher and you are interested in figuring out behavior from the methods section of scientific papers."
)
section = methods_text
section = pup_retrieval_text 
prompt = (
    f"{instructions_prompt} \n"
    f"Here is the section method of a paper {section} \n" 
    "List all the behaviors described in the section using complete sentences and use definitions from the neurobehavior ontology."
    "Include only description of animal behavior and not of the experimental setup. Do not add jump lines between the items in the list or any other text. "
    "If no behaviors are described, please write 'None'."
)

response = extract_behavior_descriptions_from_prompt(prompt)

In [101]:
behaviors = response.split("\n")
behaviors = [b.split("-")[1].strip() for b in behaviors if b]
behaviors

['Pup retrieval: The behavior of the virgin female mice retrieving displaced pups and returning them back to the nest within a given time frame.',
 'Acclimatization: The process of allowing the female mice to adjust and become familiar with the behavioral arena before each testing session.',
 'Nesting: The act of covering the pups with nesting material in a corner of the arena.',
 'Pup displacement: The action of removing a pup from the nest and placing it in an opposite corner of the arena.',
 'Retrieval success: The outcome of a trial where the female mouse successfully retrieves the displaced pup and returns it back to the nest within the allotted time.',
 'Retrieval failure: The outcome of a trial where the female mouse fails to retrieve the displaced pup within the allotted time, resulting in the pup being returned to the nest.',
 'Vocalization: The production of vocal sounds by the isolated pups during testing, verified using an ultrasonic microphone.']

In [102]:
from utils.beahvior_meatdata_extraction import ground_metadata_in_ontologies

terms_list = behaviors
term_to_identifiers_dict = ground_metadata_in_ontologies(term_list=terms_list)
term_to_identifiers_dict

{'NBO:0000777': {'name': 'vocalization behavior phenotype',
  'score': 0.86675507,
  'url': 'http://purl.obolibrary.org/obo/NBO_0000777',
  'context': 'Vocalization: The production of vocal sounds by the isolated pups during testing, verified using an ultrasonic microphone.'},
 'NBO:0000157': {'name': 'nest building behavior',
  'score': 0.854887,
  'url': 'http://purl.obolibrary.org/obo/NBO_0000157',
  'context': 'Nesting: The act of covering the pups with nesting material in a corner of the arena.'},
 'NBO:0000155': {'name': 'offspring retrieval',
  'score': 0.83897096,
  'url': 'http://purl.obolibrary.org/obo/NBO_0000155',
  'context': 'Retrieval failure: The outcome of a trial where the female mouse fails to retrieve the displaced pup within the allotted time, resulting in the pup being returned to the nest.'},
 'NBO:0000125': {'name': 'submissive behavior towards male mice',
  'score': 0.82962805,
  'url': 'http://purl.obolibrary.org/obo/NBO_0000125',
  'context': 'Acclimatization